# Comparing alignment-free and usual MSA-based phylogenetic matrics

## Constructing our dataset of alpha, beta, delta and gammacoronaviruses

### Downloading genomic sequences from NCBI via custon E-utilities custom link
Sequences obtained following https://doi.org/10.1371/journal.pone.0264640.
Use it in a simple web-browser.

https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nucleotide&id=NC_022103.1,NC_048216.1,NC_046964.1,NC_032107.1,NC_028833.1,NC_028824.1,NC_028814.1,NC_028811.1,NC_032730.1,NC_018871.1,NC_025217.1,NC_030886.1,MN996532,NC_034440.1,NC_014470.1,NC_010438.1,NC_010437.1,NC_009988.1,NC_009021.1,NC_009020.1,NC_009019.1,NC_009657.1,NC_005831.2,NC_002645.1,NC_045512.2,NC_006213.1,NC_038294.1,NC_019843.3,NC_006577.2,NC_004718.3,NC_034972.1,NC_028752.1,MT121216.1,MT040336.1,NC_039207.1,NC_026011.1,NC_017083.1,NC_012936.1,NC_003045.1,KX432213.1,JX860640.1,NC_010646.1,NC_038861.1,NC_030292.1,NC_028806.1,NC_023760.1,NC_002306.3,NC_003436.1,NC_039208.1,NC_010800.1,NC_048214.1,NC_048213.1,NC_046965.1,NC_001451.1,NC_011547.1,NC_016992.1,NC_016991.1,NC_016996.1,NC_016995.1,NC_016994.1,NC_016993.1,NC_011550.1,NC_011549.1,NC_001846.1,NC_048217.1,AC_000192.1&rettype=fasta


Then, place the file sequences in the notebook's folder in order to facilitate finding this file.
(We named this file as cov4gen.fasta)
__________________________________________________________________________________________________________

Verifying its content:

In [1]:
!grep -c ">" cov4gen.fasta

66


In [2]:
!grep ">" cov4gen.fasta

>NC_022103.1 Bat coronavirus CDPHE15/USA/2006, complete genome
>NC_048216.1 NL63-related bat coronavirus strain BtKYNL63-9b, complete genome
>NC_046964.1 Alphacoronavirus Bat-CoV/P.kuhlii/Italy/3398-19/2015, complete genome
>NC_032107.1 NL63-related bat coronavirus strain BtKYNL63-9a, complete genome
>NC_028833.1 BtNv-AlphaCoV/SC2013, complete genome
>NC_028824.1 BtRf-AlphaCoV/YN2012, complete genome
>NC_028814.1 BtRf-AlphaCoV/HuB2013, complete genome
>NC_028811.1 BtMr-AlphaCoV/SAX2011, complete genome
>NC_032730.1 Lucheng Rn rat coronavirus isolate Lucheng-19, complete genome
>NC_018871.1 Rousettus bat coronavirus HKU10, complete genome
>NC_025217.1 Bat Hp-betacoronavirus/Zhejiang2013, complete genome
>NC_030886.1 Rousettus bat coronavirus isolate GCCDC1 356, complete genome
>MN996532.2 Bat coronavirus RaTG13, complete genome
>NC_034440.1 Bat coronavirus isolate PREDICT/PDF-2180, complete genome
>NC_014470.1 Bat coronavirus BM48-31/BGR/2008, complete genome
>NC_010438.1 Miniopterus ba

We add these three sequences from GISAID, as described in the method from the above paper:
* Beijing.IVDC_01
* Beijing.IVDC_02
* Beijing.IVDC_03

In [3]:
!cat cov4gen.fasta 1673264835172.sequences.fasta > db_cov.fasta

In [4]:
!grep -c ">" db_cov.fasta

69


### Multiple Sequence Analysis
Alignment of the 69 sequences using mafft linsi method. 
(We do not recommend to run this in a common user PC because of its wide RAM usage).
This command above consumed high ammounts of RAM!!! (+to 60 GB)

In [ ]:
!nohup linsi --thread 32 --leavegappyregion db_cov.fasta > alig_cov.fasta &

Before running IQTREE, we trimmed the alignment with trimAl v1.4.rev22. We choose to remove all positions in the alignment with gaps in 10% or more of the sequences, unless this leaves less than 70% of original alignment.

In [ ]:
!trimal -in alig_cov.fasta -out trimm_alig_cov.fasta -gt 0.9 -cons 70

### Estimating Maximum-Likelihood Phylogeny
The maximum-likelihood phylogenetic trees were obtained using IQTREE v2.2.0 software with the Ultrafast Bootstrap parameter set at 1000 and 1000 replicates of the SH-aLRT test. We used the GTR+I+G substitution model, as there is evidence that this model choice leads to phylogenetic analyses comparable to those made using current strategies for evolutionary model selection.

In [ ]:
%%bash
IQTREE2=/dados/software/iqtree-2.2.0-Linux/bin/iqtree2
$IQTREE2 -T AUTO -s trimm_alig_cov.fasta -m GTR+I+G -B 1000 -alrt 1000

## Installing used R packages
### One must properly install rpy2 before continue
https://rpy2.github.io/doc/latest/html/overview.html#install-installation

In [1]:
%load_ext rpy2.ipython

Instalation

In [10]:
%%R
# Install and load the Bioconductor manager
if (!requireNamespace("BiocManager", quietly = TRUE))
  install.packages("BiocManager")
BiocManager::install()

# Install the required Bioconductor packages
BiocManager::install(c("phyloTop", "treeio", "castor", "ggtree"))

# Install the required CRAN packages
install.packages(c("tidyverse", "ape", "ggtree", "tidytree", "ggpubr"))



R[write to console]: 'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cloud.r-project.org

R[write to console]: Bioconductor version 3.15 (BiocManager 1.30.20), R 4.2.2 Patched (2022-11-10
  r83330)

R[write to console]: Installation paths not writeable, unable to update packages
  path: /usr/lib/R/library
  packages:
    boot, foreign, Matrix, mgcv, spatial

R[write to console]: 'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cloud.r-project.org

R[write to console]: Bioconductor version 3.15 (BiocManager 1.30.20), R 4.2.2 Patched (2022-11-10
  r83330)

R[write to console]: Installation paths not writeable, unable to update packages
  path: /usr/lib/R/library
  packages:
    boot, foreign, Matrix, mgcv, spatial

R[write to console]: Ins

gcc -I"/usr/share/R/include" -DNDEBUG  -I'/home/murilo/R/x86_64-pc-linux-gnu-library/4.2/Rcpp/include'    -fpic  -g -O2 -ffile-prefix-map=/build/r-base-Faorqz/r-base-4.2.2.20221110=. -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2  -c BIONJ.c -o BIONJ.o
gcc -I"/usr/share/R/include" -DNDEBUG  -I'/home/murilo/R/x86_64-pc-linux-gnu-library/4.2/Rcpp/include'    -fpic  -g -O2 -ffile-prefix-map=/build/r-base-Faorqz/r-base-4.2.2.20221110=. -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2  -c NNI.c -o NNI.o
g++ -std=gnu++14 -I"/usr/share/R/include" -DNDEBUG  -I'/home/murilo/R/x86_64-pc-linux-gnu-library/4.2/Rcpp/include'    -fpic  -g -O2 -ffile-prefix-map=/build/r-base-Faorqz/r-base-4.2.2.20221110=. -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2  -c RcppExports.cpp -o RcppExports.o
gcc -I"/usr/share/R/include" -DNDEBUG  -I'/home/murilo/R/x86_64-pc-linux-gnu-library/4.2/

installing to /home/murilo/R/x86_64-pc-linux-gnu-library/4.2/00LOCK-ape/00new/ape/libs
** R
** data
** inst
** byte-compile and prepare package for lazy loading
** help
*** installing help indices
** building package indices
** installing vignettes
** testing if installed package can be loaded from temporary location
** checking absolute paths in shared objects and dynamic libraries
** testing if installed package can be loaded from final location
** testing if installed package keeps a record of temporary installation path
* DONE (ape)
* installing *source* package ‘ggpubr’ ...
** package ‘ggpubr’ successfully unpacked and MD5 sums checked
** using staged installation
** R
** data
*** moving datasets to lazyload DB
** inst
** byte-compile and prepare package for lazy loading
** help
*** installing help indices
** building package indices
** testing if installed package can be loaded from temporary location
** testing if installed package can be loaded from final location
** testing if

Load the installed packages

In [11]:
%%R
library(phyloTop)
library(treeio)
library(castor)
library(tidyverse)
library(ape)
library(ggtree)

R[write to console]: Error: package or namespace load failed for ‘tidyverse’:
 .onAttach failed in attachNamespace() for 'tidyverse', details:
  call: value[[3L]](cond)
  error: Package ‘ggplot2’ version 3.4.0 cannot be unloaded:
 Error in unloadNamespace(package) : namespace ‘ggplot2’ is imported by ‘patchwork’, ‘aplot’, ‘ggplotify’, ‘ggtree’, ‘ggfun’, ‘ggpmisc’, ‘ggpp’ so cannot be unloaded

R[write to console]: In addition: 
R[write to console]: Warning messages:

R[write to console]: 1: package(s) not installed when version(s) same as or greater than current; use
  `force = TRUE` to re-install: 'phyloTop' 'treeio' 'castor' 'ggtree' 

R[write to console]: 2: package ‘ggtree’ is not available for this version of R

A version of this package for your version of R might be available elsewhere,
see the ideas at
https://cran.r-project.org/doc/manuals/r-patched/R-admin.html#Installing-packages 




Error: package or namespace load failed for ‘tidyverse’:
 .onAttach failed in attachNamespace() for 'tidyverse', details:
  call: value[[3L]](cond)
  error: Package ‘ggplot2’ version 3.4.0 cannot be unloaded:
 Error in unloadNamespace(package) : namespace ‘ggplot2’ is imported by ‘patchwork’, ‘aplot’, ‘ggplotify’, ‘ggtree’, ‘ggfun’, ‘ggpmisc’, ‘ggpp’ so cannot be unloaded


RInterpreterError: Failed to parse and evaluate line 'library(phyloTop)\nlibrary(treeio)\nlibrary(castor)\nlibrary(tidyverse)\nlibrary(ape)\nlibrary(ggtree)\n'.
R error message: "Error: package or namespace load failed for ‘tidyverse’:\n .onAttach failed in attachNamespace() for 'tidyverse', details:\n  call: value[[3L]](cond)\n  error: Package ‘ggplot2’ version 3.4.0 cannot be unloaded:\n Error in unloadNamespace(package) : namespace ‘ggplot2’ is imported by ‘patchwork’, ‘aplot’, ‘ggplotify’, ‘ggtree’, ‘ggfun’, ‘ggpmisc’, ‘ggpp’ so cannot be unloaded"